# Try MultiDataHandler class

In [25]:
import os
import pandas as pd
import numpy as np
import importlib
import matplotlib.pyplot as plt

In [167]:
import anomalydetector
import anomalydetector.multidata_handler
import anomalydetector.utils
import anomalydetector.detector_model
import solardatatools

importlib.reload(anomalydetector)
importlib.reload(anomalydetector.multidata_handler)
importlib.reload(anomalydetector.utils)
importlib.reload(anomalydetector.detector_model)

from anomalydetector.multidata_handler import MultiDataHandler,train_test_split
from solardatatools.algorithms import Dilation
from anomalydetector.utils import common_days,form_xy,reshape_for_cheb,optimal_weighted_regression
from anomalydetector.detector_model import QLinear

In [27]:
folder_path = "C:/Users/coren/Solar-data-tools/site_data"

dfs = {}
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        dfs[filename] = pd.read_csv(file_path)
        dfs[filename]['measured_on'] = pd.to_datetime(dfs[filename]['measured_on'])+pd.Timedelta(hours=-8)
        dfs[filename] = dfs[filename].iloc[:,:2]

In [28]:
dhs = MultiDataHandler(data_frames=list(dfs.values()),datetime_col='measured_on')

In [29]:
dhs.align()

Aligning datasets: 100%|██████████| 9/9 [00:00<00:00, 239.48it/s]


In [30]:
dhs.dilate(ndil = 51)

In [168]:
dhs_train,dhs_test = train_test_split(dhs,test_size=0.1,shuffle=True)

In [159]:
sites = dhs.keys

In [152]:
m1 = QLinear(sites,51,'ac_power_inv_30219')

In [153]:
m1.fit_quantiles(dhs,param={'solver_quantiles' : 'mosek'})

100%|███████████████████████████████████████████| 10/10 [01:59<00:00, 11.94s/it]
c:\Users\coren\Solar-data-tools\environments\ra_dev\Lib\site-packages\spcqe\extrapolate_asymptotic.py:61: RuntimeWarning: invalid value encountered in multiply
  return yasympt + alpha * exp_output
100%|███████████████████████████████████████████| 10/10 [01:59<00:00, 11.92s/it]


In [ ]:
m1.fit_linear({'nlag' : 3, 'num_basis' : 8})

In [158]:
m1.fit_residuals()

{'accuracy_train': 0.9075215782983971,
 'f1_score_train': 0.8979591836734694,
 'accuracy_test': 0.8620689655172413,
 'f1_score_test': 0.8585858585858586}

In [144]:
m1.quantile_train[m1.target][3:47].flatten()

array([-0.20089628, -0.28794225, -0.1488956 , ...,  1.41510804,
        1.51060023, -0.25265516], shape=(22308,))